In [ ]:
import informalean.data.statements.process as process

In [ ]:
from .autonotebook import tqdm as notebook_tqdm

In [ ]:
import informalean.config as config

In [ ]:
config = config.load_config("cloud")

In [ ]:
processed = process.load_processed_statements(config)

In [ ]:
val_with_len = processed["val"].map(lambda e: {"prompt_len": len(e["prompt"][1]["content"])})

In [ ]:
val_with_len_sorted = val_with_len.sort("prompt_len", reverse=True)

In [ ]:
val_with_len_sorted[2]

In [ ]:
from transformers import AutoModelForCausalLM
model0 = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-7B-Instruct", device_map="cuda")

In [ ]:
from transformers import AutoTokenizer
tok0 = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")

In [ ]:
def infer(model, tok, prompt):
        inputs = tok.apply_chat_template(prompt, tokenize=True, return_dict=True, add_generation_prompt=True, return_tensors="pt")
        inputs.to(model.device)
        with torch.no_grad():
            out = model.generate(**inputs, max_new_tokens=512, do_sample=False)
        return tok.decode(out[0])

In [ ]:
model0.eval()

In [ ]:
infer(model0, tok0, val_with_len_sorted[1000]["prompt"])

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from informalean.train.statements.test_inference import TestInference
ti = TestInference("/workspace/informalean/remote-artifacts/statements/020726_0/", config.train)